In [1]:
import pandas as pd
import numpy as np

In [2]:
df_log=pd.read_csv("all_log_sorted.csv", dtype=object)

In [3]:
df_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63668283 entries, 0 to 63668282
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   time         object
 1   user_id      object
 2   creative_id  object
 3   click_times  object
dtypes: object(4)
memory usage: 1.9+ GB


In [4]:
sentences = df_log.groupby(['user_id']).apply(lambda x: x['creative_id'].tolist()).tolist()

In [7]:
!pip install gensim
from gensim.models import Word2Vec

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 24.2 MB 5.1 MB/s eta 0:00:01MB/s eta 0:00:09
     |████████████████████████████████| 103 kB 87.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 89.7 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 90.9 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 71.0 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 88.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 73.8 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 70.6 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=6518a43f03734eecdf9faed9f9695a39df7d0a966181b669ac79592fd283109c
  Stored in directory: /home/tione/.cache/pip/wheels/54/7f/ad/18624c030ede84ebb5f4bcd3a6cdfab71662eccc4f4042df14
Successfully built smart-open
  

In [8]:
import time
start = time.clock()
model = Word2Vec(sentences=sentences, min_count=1, sg=1, window=10, size=128, workers=25, seed=2020, iter=5)
end = time.clock()
print('Running time: %s Seconds'%(end-start))
model.save("Word2vec_128.w2v")

Running time: 14292.419797 Seconds


In [10]:
model = Word2Vec.load("Word2vec_128.w2v").wv

In [28]:
# train_X = np.array(df_log.groupby(['user_id']).apply(lambda x: model[x['creative_id']]))

In [11]:
train=pd.read_csv("train_all.csv")

In [12]:
import sys,time
user_list=[]
for n in range(0,900000):
    sum=0
    item_list=train["user_items"].loc[n].split(" ")
    for i in item_list:
        sum+=model[i]
    user_list.append((sum/len(item_list)).tolist())
    
    sys.stdout.write('\r%s%%'%((n+1)*100/900000))
    sys.stdout.flush()

100.0%88888888889%%%%%%

In [13]:
em=pd.DataFrame(user_list)
em

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.764329,0.289069,-0.183351,0.080797,0.011061,-0.370083,-0.014335,0.187192,0.033921,0.004573,...,-0.206492,-0.190564,-0.365125,0.207170,-0.284092,-0.090711,0.097633,0.353195,0.186528,-0.259756
1,-0.503267,0.074858,-0.444321,-0.166103,-0.220198,0.011181,-0.204826,-0.143862,0.324566,0.192023,...,-0.062863,-0.095127,-0.857622,-0.179263,-0.500010,0.132548,0.211456,0.293390,0.121368,-0.109452
2,-0.418707,0.239730,-0.401998,-0.027175,0.041496,-0.431849,-0.188218,0.011419,0.294402,-0.020328,...,-0.137102,-0.543590,-0.435122,0.122904,-0.405559,-0.070651,0.223165,0.125867,0.443155,-0.338863
3,-0.552039,0.132106,-0.358989,-0.264487,0.009757,-0.310496,-0.215398,-0.213641,0.430637,-0.061179,...,-0.236109,-0.273338,-0.541987,0.128642,-0.300143,0.002930,0.219131,0.071365,0.451295,-0.131015
4,-0.418063,0.265463,-0.386961,0.010197,0.152503,-0.326929,-0.132815,0.194857,0.056925,-0.207085,...,-0.340586,-0.179840,-0.336602,0.151279,-0.341013,0.133223,0.074367,0.072279,0.256254,-0.214060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,-0.482918,0.395967,-0.214539,-0.201395,-0.098439,-0.410575,-0.161701,-0.074089,0.098357,-0.011478,...,-0.205033,-0.368773,-0.483162,0.030605,-0.205968,-0.063473,0.113490,-0.041018,0.300693,-0.104157
899996,-0.783242,0.216547,-0.446433,-0.239602,-0.072906,-0.483764,-0.616297,-0.155411,-0.046005,0.051443,...,0.287972,-0.506266,-0.478922,-0.082352,-0.125860,-0.285319,0.424066,0.072593,0.091019,-0.143212
899997,-0.635579,0.087936,-0.411182,-0.393224,0.039037,-0.763754,-0.370149,0.014437,0.296924,0.008270,...,-0.296481,-0.468750,-0.502048,0.165207,-0.349062,-0.113864,0.001046,-0.022316,-0.053274,-0.419154
899998,-0.716420,0.185160,-0.532090,0.098736,0.149245,-0.573841,-0.357730,-0.302757,0.154937,-0.139393,...,0.272161,-0.332018,-0.523237,0.066642,-0.358325,-0.234780,0.092732,0.205344,-0.108675,-0.074199


In [14]:
em.to_csv("w2v_train_128.csv",index=False)

In [15]:
test=pd.read_csv("test_all.csv")

In [16]:
import sys,time
user_list=[]
for n in range(0,1000000):
    sum=0
    item_list=test["user_items"].loc[n].split(" ")
    for i in item_list:
        sum+=model[i]
    user_list.append((sum/len(item_list)).tolist())
    
    sys.stdout.write('\r%s%%'%((n+1)*100/1000000))
    sys.stdout.flush()

100.0%9%

In [17]:
em=pd.DataFrame(user_list)
em

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.371500,0.520530,-0.504535,-0.226916,-0.344244,-0.666416,-0.306514,0.163879,0.053559,0.013212,...,-0.492837,-0.359873,-0.587879,0.172495,-0.473458,0.135245,-0.165464,-0.064705,0.330927,-0.062600
1,-0.652399,0.058127,-0.491925,-0.023807,0.278834,-0.491605,-0.131637,-0.273791,0.090128,-0.094283,...,0.244137,-0.575985,-0.336386,0.029982,-0.376248,-0.190995,0.485145,-0.144279,0.241411,-0.225490
2,-0.680846,0.165554,-0.384081,-0.235787,-0.172647,-0.214824,-0.256202,-0.096142,0.165658,0.173328,...,0.025466,-0.151416,-0.763351,-0.090224,-0.332376,0.082717,0.391688,0.339295,0.115779,-0.312235
3,-0.192725,0.215610,-0.507664,-0.304545,-0.098006,-0.128032,-0.377013,-0.093387,0.069011,0.330999,...,-0.056040,-0.338795,-0.813904,-0.272950,-0.148837,-0.030100,0.013114,0.446035,-0.181901,-0.253249
4,-0.434629,0.197581,-0.486156,-0.034725,0.092131,-0.738370,-0.068815,-0.332544,0.123878,-0.053949,...,-0.090317,-0.380860,-0.373552,-0.026189,-0.280841,0.172646,0.035474,0.082843,0.089133,-0.253863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.438514,0.180160,-0.362428,-0.019102,-0.228293,-0.555060,-0.183594,-0.175420,0.083503,0.104564,...,-0.189326,-0.393977,-0.530039,0.090548,-0.302253,-0.012152,0.037638,-0.114731,0.363962,-0.340434
999996,-0.594284,0.345622,-0.470185,-0.053193,-0.020274,-0.312803,-0.239889,0.245528,0.201295,0.045599,...,-0.016962,-0.156381,-0.318134,-0.044234,-0.324895,0.103303,0.214852,0.271184,0.363854,-0.317056
999997,-0.284347,0.174410,-0.680575,-0.101008,-0.070435,-0.515805,-0.217219,0.138421,-0.096977,-0.128143,...,-0.316307,-0.068609,-0.208443,0.321354,-0.258218,-0.068479,0.222186,0.167668,0.144046,-0.096395
999998,-0.174377,0.348553,-0.490484,-0.161945,-0.077104,-0.457671,-0.120463,0.232689,0.215962,-0.118729,...,-0.255217,-0.149870,-0.251665,0.033057,-0.292419,-0.041865,0.053752,0.174418,0.330676,-0.104600


In [18]:
em.to_csv("w2v_test_128.csv",index=False)